In [28]:
import torch
import torch.nn as nn
import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def vgg_block(num_convs, in_channels, out_channels):
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels,
                                 kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels,
                                 kernel_size=3, padding=1))
        blk.append(nn.ReLU())
    # 这里会使宽度减半
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*blk)

In [29]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 经过5个vgg_block，宽高会减半5次，变成224/2**5=224/32=7
# c*h*w
fc_features = 512 * 7 * 7
fc_hidden_units = 4096

In [30]:
def vgg(conv_arch, fc_features, fc_hidden_units=4096):
    net = nn.Sequential()
    # 卷积层部分
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        # 每经过一个vgg_block都会使宽高减半
        net.add_module('vgg_block_' + str(i + 1), vgg_block(num_convs, in_channels, out_channels))
    # 全连接部分
    net.add_module('fc', nn.Sequential(d2l.FlattenLayer(), nn.Linear(fc_features, fc_hidden_units),
                                       nn.ReLU(),
                                       nn.Dropout(0.5),
                                       nn.Linear(fc_hidden_units, fc_hidden_units), nn.ReLU(),
                                       nn.Dropout(0.5), nn.Linear(fc_hidden_units, 10)
                                       ))
    return net


In [31]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224)
# named_children获取一级子模块及其名字
# named_modules会返回所有子模块，包括子模块的子模块
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape: ', X.shape)

vgg_block_1 output shape:  torch.Size([1, 64, 112, 112])
vgg_block_2 output shape:  torch.Size([1, 128, 56, 56])
vgg_block_3 output shape:  torch.Size([1, 256, 28, 28])
vgg_block_4 output shape:  torch.Size([1, 512, 14, 14])
vgg_block_5 output shape:  torch.Size([1, 512, 7, 7])
fc output shape:  torch.Size([1, 10])


In [38]:
ratio = 4
small_conv_arch = ((1, 1, 64 // ratio),
                   (1, 64 // ratio, 128 // ratio),
                   (2, 128 // ratio, 256 // ratio),
                   (2, 256 // ratio, 512 // ratio),
                   (2, 512 // ratio, 512 // ratio))
net = vgg(small_conv_arch, fc_features // ratio, fc_hidden_units // ratio)

In [39]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size,
                                                    root='F:\python_code\pycharm_project\DiveIntoDL\chapter2\data\FashionMNIST',
                                                    resize=224)
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.8464, train acc 0.676, test acc 0.865, time 1348.3 sec
epoch 2, loss 0.3144, train acc 0.885, test acc 0.894, time 1345.9 sec
epoch 3, loss 0.2648, train acc 0.903, test acc 0.899, time 1330.3 sec
epoch 4, loss 0.2293, train acc 0.917, test acc 0.913, time 1340.2 sec
epoch 5, loss 0.2111, train acc 0.922, test acc 0.914, time 1331.5 sec
